# Generating the Clinical Trials Datasets

## *1. Select all gene signatures of compounds in the LINCS dataset*

This was done by selecting the gene signatures that were:
1. Treated with a small compound
2. Treated by one of the top three cell lines used (A375, MCF7, or PC3)

To remove duplicate gene signatures treated with the same compound, only the gene signature with the highest TAS score was kept.

In [1]:
# import data manipulation tool
import pandas as pd

In [2]:
# read in the LINCS signatures
file_path = "./ref_data/GSE70138_Broad_LINCS_sig_info.txt"
LINCS_sigs = pd.read_csv(file_path, index_col=0, sep="\t")
# keep only the signatures treated with small compounds
LINCS_sigs = LINCS_sigs[LINCS_sigs["pert_type"] == "trt_cp"]
# keep only the signatures treated by A375, MCF7, or PC3
cell_lines = ["A375", "MCF7", "PC3"]
LINCS_sigs = LINCS_sigs[LINCS_sigs["cell_id"].isin(cell_lines)]
LINCS_sigs.head()

,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
sig_id,,,,,,,
LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...
LJP005_A375_24H:A08,BRD-K76908866,CP-724714,trt_cp,A375,3.33 um,24 h,LJP005_A375_24H_X1_B19:A08|LJP005_A375_24H_X2_...
LJP005_A375_24H:A09,BRD-K76908866,CP-724714,trt_cp,A375,1.11 um,24 h,LJP005_A375_24H_X1_B19:A09|LJP005_A375_24H_X2_...
LJP005_A375_24H:A10,BRD-K76908866,CP-724714,trt_cp,A375,0.37 um,24 h,LJP005_A375_24H_X1_B19:A10|LJP005_A375_24H_X2_...
LJP005_A375_24H:A11,BRD-K76908866,CP-724714,trt_cp,A375,0.12 um,24 h,LJP005_A375_24H_X1_B19:A11|LJP005_A375_24H_X2_...


In [3]:
# read in the metrics file containing the tas scores of gene signatures
file_path = "./ref_data/GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt"
sig_metrics = pd.read_csv(file_path, index_col="sig_id", sep="\t")
# store tas scores of the sig ids treated with compounds
tas = sig_metrics.loc[LINCS_sigs.index, "tas"]
# add the tas scores to signature information
LINCS_sigs = pd.concat([LINCS_sigs, tas], axis=1)
LINCS_sigs.head()

,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id,tas
sig_id,,,,,,,,
LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...,0.442109
LJP005_A375_24H:A08,BRD-K76908866,CP-724714,trt_cp,A375,3.33 um,24 h,LJP005_A375_24H_X1_B19:A08|LJP005_A375_24H_X2_...,0.458614
LJP005_A375_24H:A09,BRD-K76908866,CP-724714,trt_cp,A375,1.11 um,24 h,LJP005_A375_24H_X1_B19:A09|LJP005_A375_24H_X2_...,0.114044
LJP005_A375_24H:A10,BRD-K76908866,CP-724714,trt_cp,A375,0.37 um,24 h,LJP005_A375_24H_X1_B19:A10|LJP005_A375_24H_X2_...,0.229162
LJP005_A375_24H:A11,BRD-K76908866,CP-724714,trt_cp,A375,0.12 um,24 h,LJP005_A375_24H_X1_B19:A11|LJP005_A375_24H_X2_...,0.188851


In [4]:
# sort values the signatures from highest to lowest tas scores
LINCS_sigs = LINCS_sigs.sort_values(by="tas", ascending=False)
# remove duplicate signatures treated with same compound and cell line by keeping the one with the highest tas
LINCS_sigs = LINCS_sigs.drop_duplicates(subset=["pert_iname", "cell_id"], keep="first")
# remove unnessary columns
cols2keep = ["pert_id", "pert_iname", "cell_id", "tas"]
LINCS_sigs = LINCS_sigs[cols2keep]
LINCS_sigs.head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A002_A375_24H:G18,BRD-K50691590,bortezomib,A375,0.870794
REP.A002_A375_24H:K11,BRD-K60230970,MG-132,A375,0.868866
REP.A024_A375_24H:N14,BRD-K78659596,ixazomib,A375,0.853718
REP.A013_A375_24H:D08,BRD-K37890730,camptothecin,A375,0.853287
REP.A002_A375_24H:L19,BRD-K17743125,belinostat,A375,0.850827


## *2. Select all gene signatures of compounds in AACT database (clinical trials)*

In [5]:
# import the clinical indications of drugs
file_path = "./ref_data/drug_ind/normalized/collapsed_LINCS_clinical_ind (AACT).txt"
LINCS_clin_ind = pd.read_csv(file_path)
# set index as the name of compound
LINCS_clin_ind = LINCS_clin_ind.set_index("pert_iname")
LINCS_clin_ind.head()

,indication
pert_iname,
2-methoxyestradiol,"Multiple Myeloma|Neoplasms, Plasma Cell|Gliobl..."
"3,3'-diindolylmethane",Breast Neoplasms|Prostatic Neoplasms|Thyroid D...
5-aminolevulinic-acid,"Bowen's Disease|Keratosis, Actinic|Keratosis|B..."
ACY-1215,"Multiple Myeloma|Neoplasms, Plasma Cell|Fallop..."
ASA-404,Neoplasms|Lung Neoplasms|Small Cell Lung Carci...


In [6]:
# keep only gene signatures treated with compounds that have an indication
shared_cps = LINCS_clin_ind.index
LINCS_clin_sigs = LINCS_sigs[LINCS_sigs["pert_iname"].isin(shared_cps)]
LINCS_clin_sigs.head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A024_A375_24H:N14,BRD-K78659596,ixazomib,A375,0.853718
REP.A002_A375_24H:L19,BRD-K17743125,belinostat,A375,0.850827
REP.A025_A375_24H:L10,BRD-K76674262,homoharringtonine,A375,0.837631
REP.A010_A375_24H:D19,BRD-K12867552,PCI-24781,A375,0.824658
REP.A007_A375_24H:G22,BRD-K52020312,metronidazole,A375,0.823728


In [7]:
# number of compounds with experimental indications
LINCS_clin_sigs.groupby("cell_id").size()

cell_id
A375    624
MCF7    624
PC3     624
dtype: int64

In [8]:
# initialize a dictionary to store all the gene signatures separated by cell line
clin_sigs = {}

# for each each cell
for cell_line in cell_lines:
    # store the gene signatures treated by that cell line in dictionary
    clin_sigs[cell_line] = LINCS_clin_sigs[LINCS_clin_sigs["cell_id"] == cell_line]

clin_sigs["MCF7"].head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A020_MCF7_24H:L01,BRD-A45889380,mepacrine,MCF7,0.806568
REP.A022_MCF7_24H:A19,BRD-K77908580,entinostat,MCF7,0.806074
REP.A010_MCF7_24H:I02,BRD-K13646352,midostaurin,MCF7,0.799386
REP.A025_MCF7_24H:L10,BRD-K76674262,homoharringtonine,MCF7,0.799271
REP.A015_MCF7_24H:P18,BRD-K61397605,romidepsin,MCF7,0.795148


## *2. Calculate the spearman correlation across landmark genes between all compounds of interest*

DRH datasets will include:
1. A pair of drugs
2. The TAS of each drug
3. The spearman correlation between their gene signatures
4. Whether the two drugs share an indication (based on data available from the Drug Repurposing Hub)

In [9]:
# read in the gene information
file_path = "./ref_data/GSE70138_Broad_LINCS_gene_info_2017-03-06.txt"
lm_genes = pd.read_csv(file_path, sep="\t", dtype=str)
# select only the gene ids of the landmark genes
lm_gene_ids = lm_genes.loc[lm_genes["pr_is_lm"] == "1", "pr_gene_id"]
# number of landmark genes
len(lm_gene_ids)

978

In [10]:
# import data manipulation packages
import cmapPy.pandasGEXpress.GCToo as GCToo
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.subset_gctoo as sg

In [11]:
### function to generate a dataframe containing spearman correlations between any pair of drugs
def gen_spearman_corrs(sigs_by_cell_line, cell_line):
    ## store the gene signatures for that cell line
    sig_info = sigs_by_cell_line[cell_line]
    ## obtain the z-scores for all landmark genes for all gene signatures
    gene_sigs = parse(LINCS_data, rid=lm_gene_ids, cid=sig_info.index)
    ## replace the signature ids with the names of the compounds
    all_cps = sig_info.loc[gene_sigs.data_df.columns, "pert_iname"]
    gene_sigs.data_df.columns = all_cps
    ## perform pairwise spearman correlation
    pairwise_corrs = gene_sigs.data_df.corr(method="spearman")
    
    ## set the index tas scores dataframe as the name of the compound
    tas_scores = sig_info.reset_index().set_index("pert_iname")
    
    ## initialize a new dataframe to store all the spearman correlations
    spearman_corrs = pd.DataFrame()
    ## iterate through all compounds
    for cp in all_cps:
        # get all the spearman correlations to the drug of interest
        cp_corrs = pairwise_corrs[cp].drop(cp).reset_index()
        # rename columns
        cp_corrs.columns = ["drug2", "spearman corr"]
        # add a column with the compound of interest
        cp_corrs.insert(0, "drug1", cp)
        # add a column with the tas of all the drug 1s
        drug1_tas = tas_scores.loc[cp, "tas"]
        cp_corrs.insert(1, "drug1 tas", drug1_tas)
        # add a column with the tas of all the drug 2s
        drug2_tas = tas_scores.loc[cp_corrs["drug2"], "tas"].tolist()
        cp_corrs.insert(3, "drug2 tas", drug2_tas)
        # add this to dataframe for all compounds
        spearman_corrs = pd.concat([spearman_corrs, cp_corrs])
        
    return spearman_corrs

In [12]:
# location of file containing the gene signatures
LINCS_data = "./ref_data/LINCS_dataset.gctx"
MCF7_corrs = gen_spearman_corrs(clin_sigs, "MCF7")
MCF7_corrs.head()

,drug1,drug1 tas,drug2,drug2 tas,spearman corr
0,aminoguanidine,0.343743,pritelivir,0.258120,0.248803
1,aminoguanidine,0.343743,aminoglutethimide,0.205099,0.312647
2,aminoguanidine,0.343743,physostigmine,0.224861,0.291091
3,aminoguanidine,0.343743,propafenone,0.309858,0.559860
4,aminoguanidine,0.343743,imiquimod,0.147057,0.253268


## *3. Generate the relevant data using data from clinical trials*

In [13]:
### function to add whether the indication is a known indication of drug1
def add_known_ind(row):
    ## get the indications of drug1
    drug1_ind = row["drug1 indications"].split("|")
    ## get the indication of drug2
    ind = row["indication"]
    
    ## return whether that indication is a known indication of drug1
    known_ind = ind in drug1_ind
    return known_ind

In [14]:
### function to add indications to spearman correlations
def gen_dataset(spearman_corrs, drug_ind):
    ## add a column with the indications for the drug 1s
    drug1_inds = drug_ind.loc[spearman_corrs["drug1"], "indication"].tolist()
    spearman_corrs.insert(2, "drug1 indications", drug1_inds)
    
    ## split the "indication" column and create a new series with duplicated rows for each indication
    ind = drug_ind["indication"].str.split("|", expand=True).stack().reset_index(level=1, drop=True).rename("indication")
    ## create a new dataframe that maps each drug to one indication
    drug2_ind = drug_ind.drop(columns="indication").join(ind).reset_index()
    ## keep only the name of drug, and the indication
    drug2_ind = drug2_ind[["pert_iname", "indication"]]
    ## create a new dataframe separated by each indication of drug 2
    train_data = pd.merge(spearman_corrs, drug2_ind, left_on="drug2", right_on="pert_iname", how="left")
    ## add a column containing whether the indication is a known indication of drug1
    train_data["known ind"] = train_data.apply(add_known_ind, axis=1)
    ## remove columns
    train_data.drop(columns=["drug1 indications", "pert_iname"], axis=1, inplace=True)
    
    return train_data

## *5. Generate clinical trials datasets for all cell lines*

In [ ]:
### initialize a dictionary to store the clinical trials datasets
clin_data = {}

### for each cell line
for cell_line in cell_lines:
    ## generate the spearman correlations for each pair of drugs based on data from clin
    clin_corrs = gen_spearman_corrs(clin_sigs, cell_line)
    ## generate the training datasets
    cell_line_data = gen_dataset(clin_corrs, LINCS_clin_ind)
    
    ## sort the rows from highest to lowest spearman correlation
    cell_line_data = cell_line_data.sort_values("spearman corr", ascending=False)
    ## remove duplicate drug-indication pairs by picking the one with the highest spearman correlation to drug2
    unique_data = cell_line_data.drop_duplicates(subset=["drug1", "indication"], keep="first")
    ## store the dataframe in the dictionary for all cell lines
    clin_data[cell_line] = unique_data
    
    ## save the dataframe for future access
    # file_path = "./DRH_clin_data/clin_data/" + cell_line + "_unique_ind.txt"
    # unique_data.to_csv(file_path, index=False)

clin_data["MCF7"].head()